# 项目背景
在现在这样一个大数据时代，新闻资讯作为获取信息的有效途径，存在信息冗杂、分类不清、表达不直观等问题，为此我们团队不断寻找创新新闻表现形式的突破口，最终发现了地图。它作为一类重要的信息载体，在信息展示方面具有直观、多维的特点。  
由此，我们尝试将新闻与地图融合，借助百度飞桨，实现新闻在地图上的可视化表达。同时，根据新闻文本内容，我们可以尝试性地探究新闻情感、主题在时空上的分布特征。

# 实现步骤

## 1、安装依赖库
- 为了将新闻与地图结合，我们需要在Jupyter中实现地图的可视化，由此我们需要使用到geopandas、mapbox-gl地图可视化组件

In [1]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!mkdir /home/aistudio/external-libraries

# 安装mapboxgl、geopandas
!pip install mapboxgl -t /home/aistudio/external-libraries
!pip install geopandas -t /home/aistudio/external-libraries
!pip install gensim -t /home/aistudio/external-libraries
!pip install pyLDAvis -t /home/aistudio/external-libraries

# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

mkdir: cannot create directory ‘/home/aistudio/external-libraries’: File exists
Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 51kB 13.3MB/s eta 0:00:01
     |████████████████████████████████| 133kB 9.2MB/s eta 0:00:01
     |████████████████████████████████| 11.6MB 14.2MB/s eta 0:00:01
     |████████████████████████████████| 2.2MB 23.2MB/s eta 0:00:01
     |████████████████████████████████| 71kB 13.4MB/s eta 0:00:01
     |████████████████████████████████| 14.5MB 8.4MB/s eta 0:00:011
     |████████████████████████████████| 235kB 23.2MB/s eta 0:00:01
     |████████████████████████████████| 1.1MB 14.6MB/s eta 0:00:01
Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 972kB 11.5MB/s eta 0:00:01
     |████████████████████████████████| 14.8MB 6.9MB/s eta 0:00:011
     |████████████████████████████████| 6.5MB 16.1MB/s eta 0:00:01
     |████████████████████████████████| 1.0MB 15.5MB/s eta 0:00:01


## 2、载入新闻数据
- 我们在数据集中提供了长江网上5月1日至5月5日的244条新闻
- 其中，“content”字段包含了新闻的正文内容

In [2]:
import pandas as pd
df_origin = pd.read_csv('data/data63541/news_origin.csv')
df_origin.head()

,_id,title,publish_time,content
0,t3621099,有一种“云”，叫“武汉云”！这个字，读懂重启后的武汉,2020-05-04 23:30,华为：“上半年在汉落地鲲鹏生态创新中心！”\n武汉：“不等6月，就在本周！”\n4月30日，...
1,t3621097,今年这一场五四晚会，让人看得泪流满面,2020-05-04 22:56,长江网5月4日讯（记者万旭明）\n看过再多遍，还是会为那些画面落泪。说过再多遍，还是想说一句...
2,t3621095,筑牢小区“防护墙”，这家物业公司上岗员工全部接受检测核酸,2020-05-04 22:43,长江网5月4日讯（记者汪甦）\n5月4日一大早，黄陂区盘龙经济开发区巢上城·俊园的业主李伟、...
3,t3621081,应勇寄语全省广大青年：凝聚青春正能量争当时代弄潮儿,2020-05-04 22:23,应勇寄语全省广大青年\n坚定理想信念 增长本领才干\n凝聚青春正能量争当时代弄潮儿\n长江网...
4,t3621078,奋斗吧青春！武汉高校青年在央视讲述武汉战疫故事,2020-05-04 22:02,长江网5月4日讯（记者杨佳峰实习记者吴金伟）\n5月4日，武汉大学、中国地质大学（武汉）等多...


## 3、对新闻文本进行情感分析
- 我们可以利用PaddleHub提供的Senta模型，输入新闻正文文本，从而对新闻进行情感倾向分析

载入PaddleHub上的预训练模型进行推理

In [3]:
import paddlehub as hub
lac = hub.Module(name="lac")
senta = hub.Module(name="senta_bilstm")

[2020-12-04 21:43:53,437] [    INFO] - Installing lac module
[2020-12-04 21:43:53,663] [    INFO] - Module lac already installed in /home/aistudio/.paddlehub/modules/lac
[2020-12-04 21:43:53,732] [    INFO] - Installing senta_bilstm module
[2020-12-04 21:43:53,737] [    INFO] - Module senta_bilstm already installed in /home/aistudio/.paddlehub/modules/senta_bilstm


- 使用Senta模型进行推理，并根据positive_probs、negative_probs计算情感得分
- 情感得分分布在(-1, 1)区间上，-1表示消极、0表示中立、1表示积极
- 将计算得到的情感得分添加到数据帧中

In [4]:
content_list = df_origin['content'].values.tolist()
content_senta_list = senta.sentiment_classify(texts=content_list, use_gpu=False)
sentiment_score = [
    content_senta_result['positive_probs'] - content_senta_result['negative_probs'] 
    for content_senta_result in content_senta_list
]
df_origin['sentiment_score'] = sentiment_score
df_origin[['sentiment_score', 'title', 'content']].head()

[2020-12-04 21:43:55,862] [    INFO] - Installing lac module
[2020-12-04 21:43:55,864] [    INFO] - Module lac already installed in /home/aistudio/.paddlehub/modules/lac


,sentiment_score,title,content
0,0.9824,有一种“云”，叫“武汉云”！这个字，读懂重启后的武汉,华为：“上半年在汉落地鲲鹏生态创新中心！”\n武汉：“不等6月，就在本周！”\n4月30日，...
1,0.9838,今年这一场五四晚会，让人看得泪流满面,长江网5月4日讯（记者万旭明）\n看过再多遍，还是会为那些画面落泪。说过再多遍，还是想说一句...
2,0.9678,筑牢小区“防护墙”，这家物业公司上岗员工全部接受检测核酸,长江网5月4日讯（记者汪甦）\n5月4日一大早，黄陂区盘龙经济开发区巢上城·俊园的业主李伟、...
3,0.9892,应勇寄语全省广大青年：凝聚青春正能量争当时代弄潮儿,应勇寄语全省广大青年\n坚定理想信念 增长本领才干\n凝聚青春正能量争当时代弄潮儿\n长江网...
4,0.9876,奋斗吧青春！武汉高校青年在央视讲述武汉战疫故事,长江网5月4日讯（记者杨佳峰实习记者吴金伟）\n5月4日，武汉大学、中国地质大学（武汉）等多...


## 4、对新闻文本进行主题分析
- 利用PaddleHub提供的词法分析，我们可以对新闻文本进行分词，进而使用LDA模型进行新闻的主题分析
- LAC词法分析的词性标签查看链接：https://www.paddlepaddle.org.cn/modelbasedetail/lac

对文本进行分词，并设置分词之后词语的选取规则，根据规则选择词语

In [5]:
def filter_word_tag(word: str, tag):
    word = word.strip()
    tag_list = [
        'v', 'vd', 'vn', 'a',
        'ad', 'an', 'd', 
    ]
    if len(word) <= 1:
        return False
    if tag not in tag_list:
        return False
    return True
    
content_lac_list = lac.cut(text=content_list, use_gpu=False, return_tag=True)
content_word_tag_filter_list = [
        [
            (word, tag) for word, tag in zip(content_lac_result['word'], content_lac_result['tag']) if filter_word_tag(word, tag)
        ]
        for content_lac_result in content_lac_list
    ]
len(content_word_tag_filter_list), content_word_tag_filter_list[0][:10]

(244,
 [('落地', 'v'),
  ('创新', 'vn'),
  ('不等', 'v'),
  ('最后', 'a'),
  ('也是', 'v'),
  ('重启', 'v'),
  ('最后', 'a'),
  ('连线', 'vn'),
  ('揭牌', 'v'),
  ('揭牌', 'v')])

生成词语列表，从而进行LDA主题模型构建

In [6]:
words_ls = [
    [word for word, tag in word_tag_list]
    for word_tag_list in content_word_tag_filter_list
]

利用gensim构建LDA模型，并输出各主题关键词

In [7]:
import gensim
from gensim import corpora, models, similarities

dictionary = corpora.Dictionary(words_ls)
corpus = [dictionary.doc2bow(words) for words in words_ls]
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=6, eval_every=None, passes=50, iterations=400)

for topic in lda.print_topics(num_words=9):
    print(topic)

2020-12-04 21:44:14,597-INFO: adding document #0 to Dictionary(0 unique tokens: [])
2020-12-04 21:44:14,634-INFO: built Dictionary(4956 unique tokens: ['一一', '一直', '上班', '上线', '不得']...) from 244 documents (total 26250 corpus positions)
2020-12-04 21:44:14,655-INFO: using symmetric alpha at 0.16666666666666666
2020-12-04 21:44:14,656-INFO: using symmetric eta at 0.16666666666666666
2020-12-04 21:44:14,657-INFO: using serial LDA version on this node
2020-12-04 21:44:14,662-INFO: running online (multi-pass) LDA training, 6 topics, 50 passes over the supplied corpus of 244 documents, updating model once every 244 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
2020-12-04 21:44:14,663-INFO: PROGRESS: pass 0, at document #244/244
2020-12-04 21:44:15,379-INFO: topic #4 (0.167): 0.012*"发展" + 0.006*"进行" + 0.006*"计算" + 0.005*"入园" + 0.005*"防控" + 0.005*"旅游" + 0.004*"服务" + 0.004*"消毒" + 0.004*"研发" + 0.004*"测量"
2020-12-04 21:44:15,380-INFO: 

(0, '0.014*"防控" + 0.011*"旅游" + 0.008*"安全" + 0.007*"出行" + 0.006*"预约" + 0.006*"预计" + 0.005*"复工" + 0.005*"进行" + 0.005*"直播"')
(1, '0.005*"广大" + 0.005*"工作" + 0.004*"已经" + 0.004*"创新" + 0.004*"防控" + 0.004*"介绍" + 0.004*"还是" + 0.004*"希望" + 0.003*"建设"')
(2, '0.015*"确诊" + 0.011*"累计" + 0.007*"恢复" + 0.007*"进行" + 0.007*"新增" + 0.006*"建设" + 0.006*"死亡" + 0.006*"介绍" + 0.006*"施工"')
(3, '0.013*"防控" + 0.009*"服务" + 0.009*"复工" + 0.007*"进行" + 0.006*"建设" + 0.006*"施工" + 0.005*"工作" + 0.005*"看到" + 0.005*"可以"')
(4, '0.025*"发展" + 0.008*"支持" + 0.008*"建设" + 0.008*"研发" + 0.008*"计算" + 0.006*"对接" + 0.006*"加大" + 0.006*"使用" + 0.006*"服务"')
(5, '0.013*"入园" + 0.011*"健康" + 0.009*"预约" + 0.009*"游玩" + 0.009*"工作" + 0.008*"进入" + 0.007*"防控" + 0.007*"测量" + 0.007*"测温"')


可视化各主题的二维平面分布

In [8]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, corpus, dictionary)

2020-12-04 21:44:20,442-INFO: Note: NumExpr detected 64 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
/home/aistudio/external-libraries/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.010831 -0.068578       1        1  24.691898
0      0.043812  0.016230       2        1  22.896641
2      0.020011 -0.118090       3        1  21.309631
5      0.126779  0.111525       4        1  15.322392
4     -0.169285  0.071828       5        1  10.251607
1     -0.032148 -0.012915       6        1   5.527833, topic_info=     Category        Freq Term       Total  loglift  logprob
term                                                        
29    Default  109.000000   发展  109.000000  30.0000  30.0000
999   Default   90.000000   确诊   90.000000  29.0000  29.0000
1050  Default   62.000000   入园   62.000000  28.0000  28.0000
120   Default   65.000000   累计   65.000000  27.0000  27.0000
1026  Default   76.000000   预约   76.000000  26.0000  26.0000
1071  Default   37.000000   游玩   37.000000  25.0000  25.0000
1067  Default  103.000000   旅游  103.000000  24.0000  24.0000
60    Default  108.000000   建设  108.000000  23.0000  23.0000
994   Default   39.000000   新增   39.000000  22.0000  22.0000
264   Default  101.000000   工作  101.000000  21.0000  21.0000
133   Default   21.000000   计算   21.000000  20.0000  20.0000
995   Default   38.000000   死亡   38.000000  19.0000  19.0000
642   Default   29.000000   研发   29.000000  18.0000  18.0000
518   Default   39.000000   测量   39.000000  17.0000  17.0000
491   Default   95.000000   健康   95.000000  16.0000  16.0000
1091  Default   35.000000   测温   35.000000  15.0000  15.0000
1858  Default   25.000000   健身   25.000000  14.0000  14.0000
515   Default   76.000000   施工   76.000000  13.0000  13.0000
998   Default   31.000000   疑似   31.000000  12.0000  12.0000
1007  Default   26.000000   回收   26.000000  11.0000  11.0000
89    Default   59.000000   支持   59.000000  10.0000  10.0000
258   Default   32.000000   加大   32.000000   9.0000   9.0000
531   Default   63.000000   进入   63.000000   8.0000   8.0000
3107  Default   27.000000   支付   27.000000   7.0000   7.0000
1414  Default   28.000000   献血   28.000000   6.0000   6.0000
57    Default   23.000000   广大   23.000000   5.0000   5.0000
20    Default   32.000000   创新   32.000000   4.0000   4.0000
52    Default   43.000000   对接   43.000000   3.0000   3.0000
2195  Default   31.000000   重生   31.000000   2.0000   2.0000
997   Default   23.000000   现有   23.000000   1.0000   1.0000
...       ...         ...  ...         ...      ...      ...
1109   Topic6    3.279467   对此    9.090278   1.8758  -6.0924
3037   Topic6    3.316703   严选    9.474520   1.8457  -6.0811
20     Topic6    6.341458   创新   32.245026   1.2691  -5.4329
331    Topic6    3.877556   坚定   15.018754   1.5413  -5.9248
978    Topic6    2.033035   热心    4.447652   2.1125  -6.5705
1234   Topic6    3.959206   办理   17.471539   1.4108  -5.9040
2547   Topic6    2.031916   获批    4.481486   2.1044  -6.5711
907    Topic6    2.032799   满怀    4.491742   2.1025  -6.5706
231    Topic6    5.243461   还是   35.551250   0.9814  -5.6231
802    Topic6    3.959074   不同   20.178457   1.2668  -5.9040
55     Topic6    5.124102   希望   37.042728   0.9173  -5.6461
185    Topic6    6.527599   已经   66.239868   0.5781  -5.4040
264    Topic6    7.295489   工作  101.799477   0.2596  -5.2928
363    Topic6    3.317047   祝福   15.311535   1.3658  -6.0810
207    Topic6    4.250478   来自   40.101036   0.6510  -5.8330
470    Topic6    4.795963   告诉   62.120884   0.3341  -5.7123
247    Topic6    5.447373   介绍  101.685692  -0.0314  -5.5849
48     Topic6    3.963807   安排   33.708359   0.7548  -5.9028
60     Topic6    5.044661   建设  108.459473  -0.1727  -5.6617
150    Topic6    5.641345   防控  243.355789  -0.8690  -5.5499
2091   Topic6    3.316890   康复   23.253035   0.9480  -6.0810
536    Topic6    3.958988   需要   72.527306  -0.0126  -5.9041
494    Topic6    3.958892   可以   77.324791  -0.0767  -5.9041
146    Topic6    3.476614   重大   35.053001   0.5846  -6.0340
11     Topic6    3.555150   也是   50.030334   0.2

计算各新闻的对应主题，并将其添加入数据帧中

In [9]:
content_topic_prob_list = list(lda[corpus])
content_topic_list = []
for content_topic_prob in content_topic_prob_list:
    max_prob = 0
    max_topic = -1
    for topic_index, topic_prob in content_topic_prob:
        if topic_prob > max_prob:
            max_prob = topic_prob
            max_topic = topic_index
    content_topic_list.append(max_topic)
df_origin['topic_index'] = content_topic_list
df_origin[['topic_index', 'title']].head()

,topic_index,title
0,2,有一种“云”，叫“武汉云”！这个字，读懂重启后的武汉
1,3,今年这一场五四晚会，让人看得泪流满面
2,3,筑牢小区“防护墙”，这家物业公司上岗员工全部接受检测核酸
3,1,应勇寄语全省广大青年：凝聚青春正能量争当时代弄潮儿
4,2,奋斗吧青春！武汉高校青年在央视讲述武汉战疫故事


## 5、提取新闻地点
- 利用PaddleHub提供的命名实体识别服务，我们可以识别出新闻中的地点名词
- 利用百度地图提供的地理编码服务，我们可以得到地名对应的经纬度

对各新闻中的地点进行地理编码

In [10]:
from trans_util import bd09_to_wgs84
import requests

news_dict = df_origin.to_dict('index')
news_list = list(news_dict.values())

for news_index, news in enumerate(news_list):
    lac_result = lac.cut(text=[news['content']], use_gpu=False, return_tag=True)[0]
    lac_zip_list = list(zip(
        lac_result['word'],
        lac_result['tag']
    ))

    location_list = [word for word, tag in lac_zip_list 
        if (('ORG' in tag) or ('LOC' in tag)) and ('武汉' != word) and ('湖北' != word) and ('中国' != word )
    ]
    
    news['locations_bd09'] = {}
    news['locations'] = {}
    for location_name in location_list:
        r = requests.get(
                    f'http://api.map.baidu.com/geocoding/v3/?city=武汉'
                    f'&address={location_name}&output=json'
                    f'&ak=xSCBGWXWcIQ5VRg1omPYWpcgtAySsMYE'
                )
        json = r.json()

        if (json['status'] is None) or (json['status'] != 0):
            continue

        result = json['result']

        if result['comprehension'] >= 70 and result['confidence'] >= 20:
            longitude, latitude = (
                result['location']['lng'], result['location']['lat'])

            news['locations_bd09'][location_name] = {
                'longitude': longitude, 'latitude': latitude}
            trans = bd09_to_wgs84(longitude, latitude)
            news['locations'][location_name] = {
                'longitude': trans[0], 'latitude': trans[1]}
    if news_index % 10 == 0:
        print(news_index)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243


将编码后的结果进行持久化存储

In [11]:
import numpy as np
np.save('work/news_geo_list.npy', news_list) 

## 6、结合情感、主题，进行新闻的地图可视化
- 利用Mapbox-gl，我们可以对生成的GeoJson进行可视化

读取地理编码后的新闻数据

In [12]:
news_list = np.load('work/news_geo_list.npy',allow_pickle='TRUE').tolist()

将新闻数据转化为数据帧格式

In [13]:
def news_list_to_location_df(news_list):
    result_list = []
    for news in news_list:
        news_location_dict = news['locations']
        for location_name in news_location_dict:
            news_copy = news.copy()
            
            news_copy['longitude'] = news_copy['locations'][location_name]['longitude']
            news_copy['latitude'] = news_copy['locations'][location_name]['latitude']

            del news_copy['locations']
            del news_copy['locations_bd09']

            result_list.append(news_copy)
    return pd.DataFrame(result_list)

df_location = news_list_to_location_df(news_list)
df_location.head()

,_id,content,latitude,longitude,publish_time,sentiment_score,title,topic_index
0,t3621099,华为：“上半年在汉落地鲲鹏生态创新中心！”\n武汉：“不等6月，就在本周！”\n4月30日，...,30.595346,114.299728,2020-05-04 23:30,0.9824,有一种“云”，叫“武汉云”！这个字，读懂重启后的武汉,2
1,t3621099,华为：“上半年在汉落地鲲鹏生态创新中心！”\n武汉：“不等6月，就在本周！”\n4月30日，...,30.495484,114.533813,2020-05-04 23:30,0.9824,有一种“云”，叫“武汉云”！这个字，读懂重启后的武汉,2
2,t3621099,华为：“上半年在汉落地鲲鹏生态创新中心！”\n武汉：“不等6月，就在本周！”\n4月30日，...,30.595346,114.299728,2020-05-04 23:30,0.9824,有一种“云”，叫“武汉云”！这个字，读懂重启后的武汉,2
3,t3621099,华为：“上半年在汉落地鲲鹏生态创新中心！”\n武汉：“不等6月，就在本周！”\n4月30日，...,30.494954,114.530962,2020-05-04 23:30,0.9824,有一种“云”，叫“武汉云”！这个字，读懂重启后的武汉,2
4,t3621099,华为：“上半年在汉落地鲲鹏生态创新中心！”\n武汉：“不等6月，就在本周！”\n4月30日，...,30.462812,114.465476,2020-05-04 23:30,0.9824,有一种“云”，叫“武汉云”！这个字，读懂重启后的武汉,2


- 将数据帧转为地理的Geojson格式，并存储
- 同时，载入geojson数据

In [20]:
from mapboxgl.utils import create_color_stops, df_to_geojson
from mapboxgl.viz import CircleViz

from mapboxgl.utils import *
from mapboxgl.viz import *

df_to_geojson(df_location, filename='work/news.geojson',
              properties=['_id', 'title', 'sentiment_score', 'topic_index'],
              lat='latitude', lon='longitude', precision=3)

import json
geodata = json.load( open( "work/news.geojson" ) )

mapbox_token = 'pk.eyJ1IjoiNDYzMTk5NDMiLCJhIjoiY2p1YjR2d2x1MDlvdDQzcHEyd21lNWZhNCJ9.SUUUXGxmlANjRuT1SmuahA'
mapbox_center =  (114.3, 30.6)
mapbox_zoom = 9

- 对新闻点进行热力可视化
- 颜色越深，代表新闻点约密集

In [21]:
heatmap_color_stops = create_color_stops([0.01, 0.25, 0.5, 0.75, 1], colors='RdPu')
heatmap_radius_stops = [[0, 5], [14, 20]] # 随着比例尺缩小，点大小增大

# 根据值进行权重设置
# color_breaks = [round(df[measure].quantile(q=x*0.1), 2) for x in range(2, 10)]
# color_stops = create_color_stops(color_breaks, colors='Spectral')
# heatmap_weight_stops = create_weight_stops(color_breaks)

# 创建热力图
viz3 = HeatmapViz(geodata, 
                  access_token=mapbox_token,
                #   weight_property="Avg Medicare Payments",
                #   weight_stops=heatmap_weight_stops,
                  color_stops=heatmap_color_stops,
                  radius_stops=heatmap_radius_stops,
                  opacity=0.8,
                  center= mapbox_center,
                  zoom=mapbox_zoom,
                  below_layer='waterway-label')

viz3.add_snapshot_links = True                    
viz3.show()

# 将可视化结果保存为文件
with open('work/heat_viz.html', 'w') as f:
    f.write(viz3.create_html())

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/IPython/core/display.py:694: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


对新闻点进行聚类可视化

In [25]:
# 创建聚类图
color_stops = create_color_stops([1, 10, 25, 50, 75, 100], colors='YlOrBr')

viz4 = ClusteredCircleViz(geodata, 
                        access_token=mapbox_token,
                        color_stops=color_stops,
                        stroke_color='black',
                        radius_stops=[[1, 5], [10, 10], [50, 15], [100, 20]],
                        radius_default=2,
                        cluster_maxzoom=10,
                        cluster_radius=30,
                        label_size=12,
                        opacity=0.9,
                        center=mapbox_center,
                        zoom=mapbox_zoom,
                    )
viz4.add_snapshot_links = True                    
viz4.show()

# 将可视化结果保存为文件
with open('work/cluster_viz.html', 'w') as f:
    f.write(viz4.create_html())

- 对情感进行可视化
- 标签代表新闻主题
- 红色代表消极情感、绿色代表积极情感、黄色为中立

In [23]:
# 生成数据颜色间隔
color_breaks = [-1,-0.5,0,0.5,1]
color_stops = create_color_stops(color_breaks, colors='RdYlGn')

# 创建地图
viz = CircleViz(geodata,
                access_token=mapbox_token,
                height='400px',
                color_property = "sentiment_score",
                color_stops = color_stops,

                radius=2.5,
                stroke_color='black',
                stroke_width=0.2,

                center = mapbox_center,
                zoom = mapbox_zoom,
                below_layer = 'waterway-label'
              )

# 将主题设置为点的标签
viz.label_property = "topic_index"
viz.stroke_width = 0
viz.label_size = 8
viz.legend_text_numeric_precision = 2

viz.add_snapshot_links = True                    
viz.show()

# 将可视化结果保存为文件
with open('work/sentiment_viz.html', 'w') as f:
    f.write(viz.create_html())

- 进行新闻主题可视化，不同的主题显示不同的颜色
- 标签代表新闻点情感得分

In [24]:
# 设置各个类别的颜色
category_color_stops = [[0, 'rgb(211,47,47)'],  
                        [1, 'rgb(81,45,168)'], 
                        [2, 'rgb(2,136,209)'], 
                        [3, 'rgb(139,195,74)'], 
                        [4, 'rgb(255,160,0)']]

# 同时设置情感得分为点的标签
viz = CircleViz(geodata, 
                access_token=mapbox_token,
                height='500px',
                label_property='sentiment_score',
                color_property='topic_index',
                color_default='grey',
                color_function_type='match',
                color_stops=category_color_stops,
                radius=2,
                center= mapbox_center,
                        zoom=mapbox_zoom)

viz.add_snapshot_links = True                    
viz.show()

# 将可视化结果保存为文件
with open('work/topic_viz.html', 'w') as f:
    f.write(viz.create_html())

# 总结
- 地图为我们提供了新闻的另一个入口，通过地图，我们可以发现新闻在空间上的分布特征，并且快速了解特定区域的新闻
- 通过百度飞桨提供的深度学习框架，我们能够快速地运用预训练模型，完成自然语言处理任务，从而对新闻文本进行分析，提取新闻地点
- 本例中以长江网上的新闻作为分析对象，对于分析全国新闻数据，需要更加完善的地点解析、编码机制，有兴趣的朋友可以参考Github中的geocoder.py文件，里面提供了处理全国范围数据的思路